In [ ]:
from rlcard.games.tractor.utils import ACTION_SPACE

s = ["\"{}\", {}".format(x, ACTION_SPACE[x]) for x in ACTION_SPACE]

s = [("{" + x + "}") for x in s]

print(', '.join(s))

In [ ]:
from rlcard.games.tractor import Dealer

dealer = Dealer('3J', None)

print(dealer.deck)
print(len(dealer.deck))

In [ ]:

import numpy as np

np_random = np.random.RandomState()

from rlcard.games.tractor import Dealer, Round, Player

players = [Player(0, np_random), Player(1, np_random), Player(2, np_random), Player(3, np_random)]

round = Round(np_random)
round.initiate(players, predefined_trump='5H')

round.current_round = [['6J'],['2S'],['NC'],['NJ']]
print(round.calc_score_in_round())


In [ ]:
from rlcard.games.tractor.utils import CARD_RANK_STR

dict = {CARD_RANK_STR[i] : i for i in range(len(CARD_RANK_STR))}

print(dict)

In [ ]:
current_hand = ['2S', '3S', '7S', 'KS', 'AS', '3H', '4H', '5H', '7H', '7H', 'AH', '2D', '4D', 'TD', 'AD', 'AD', '3J', '7J', '8J', 'JJ', 'AJ', 'AJ', 'NS', 'ND', 'NJ']
print(Judger.playable_cards_from_hand(current_hand, '9C'))

In [ ]:
from rlcard.games.tractor import Dealer, Judger
from rlcard.games.tractor.utils import TRUMP_CANDIDATE_STR

action_list = []

for trump in TRUMP_CANDIDATE_STR:
    dealer = Dealer(trump, None)
    current_hand = dealer.deck
    playable_cards = Judger.playable_cards_from_hand(current_hand, trump)
    for cards in playable_cards:
        if cards not in action_list:
            action_list.append(cards)

action_list_str = [','.join(action) for action in action_list]
action_list_str.append('pass')
action_list_str.append('pass_score')
print(action_list_str)
print()

action_dict = {action_list_str[i] : i for i in range(len(action_list_str))}
print(action_dict)


In [1]:
from rlcard.games.tractor.utils import ACTION_LIST

for action in ACTION_LIST:
    cards = action.split(',')
    if len(cards) == 4:
        continue
    if len(cards) == 1:
        pass
    
    if len(cards) == 2:
        pass


['3H', '4H', '5H', '6H', '7H', '8H', '9H', 'TH', 'JH', 'QH', 'KH', 'AH', '3C', '4C', '5C', '6C', '7C', '8C', '9C', 'TC', 'JC', 'QC', 'KC', 'AC', '3D', '4D', '5D', '6D', '7D', '8D', '9D', 'TD', 'JD', 'QD', 'KD', 'AD', '3J', '4J', '5J', '6J', '7J', '8J', '9J', 'TJ', 'JJ', 'QJ', 'KJ', 'AJ', 'NH', 'NC', 'ND', 'NJ', 'BJ', 'RJ', '3H,3H', '4H,4H', '5H,5H', '6H,6H', '7H,7H', '8H,8H', '9H,9H', 'TH,TH', 'JH,JH', 'QH,QH', 'KH,KH', 'AH,AH', '3C,3C', '4C,4C', '5C,5C', '6C,6C', '7C,7C', '8C,8C', '9C,9C', 'TC,TC', 'JC,JC', 'QC,QC', 'KC,KC', 'AC,AC', '3D,3D', '4D,4D', '5D,5D', '6D,6D', '7D,7D', '8D,8D', '9D,9D', 'TD,TD', 'JD,JD', 'QD,QD', 'KD,KD', 'AD,AD', '3J,3J', '4J,4J', '5J,5J', '6J,6J', '7J,7J', '8J,8J', '9J,9J', 'TJ,TJ', 'JJ,JJ', 'QJ,QJ', 'KJ,KJ', 'AJ,AJ', 'NH,NH', 'NC,NC', 'ND,ND', 'NJ,NJ', 'BJ,BJ', 'RJ,RJ', '3H,3H,4H,4H', '4H,4H,5H,5H', '5H,5H,6H,6H', '6H,6H,7H,7H', '7H,7H,8H,8H', '8H,8H,9H,9H', '9H,9H,TH,TH', 'TH,TH,JH,JH', 'JH,JH,QH,QH', 'QH,QH,KH,KH', 'KH,KH,AH,AH', '3C,3C,4C,4C', '4C,4C,5C